In [1]:
from model_training.cnn_classifier import CNNClassifier

In [2]:
model = CNNClassifier(
    "../models/first-milestone",
    "../data/tsrc_sprectrograms/train", 
    "../data/tsrc_sprectrograms/validation",
)

In [2]:
model.fit(epochs=10, batch_size=32)

Running training on GPU...


 10%|█         | 1/10 [01:21<12:15, 81.72s/it]

Epoch 0 Done after 81.72338390350342 seconds


 20%|██        | 2/10 [02:15<08:40, 65.02s/it]

Epoch 1 Done after 53.322589635849 seconds


...

 90%|█████████ | 9/10 [08:19<00:52, 52.95s/it]

Epoch 8 Done after 53.24813151359558 seconds


100%|██████████| 10/10 [09:08<00:00, 54.85s/it]

Epoch 9 Done after 49.46444511413574 seconds
Finished Training


Error
[0.50580114 0.49419883]
Error
[0.49711615 0.5028838 ]
Error
[0.5145472  0.48545277]
Error
[0.5079888  0.49201116]
Error
[0.4908081 0.5091919]
Error
[0.49727994 0.50272006]
Error
[0.49848995 0.50151   ]
Error
[0.47584707 0.524153  ]
Error
[0.49189025 0.5081098 ]
Error
[0.51787716 0.48212284]
Error
[0.4818158  0.51818424]
Error
[0.4770173 0.5229827]
Error
[0.51243347 0.4875665 ]
Error
[0.51150143 0.48849857]
Error
[0.48737848 0.5126215 ]
Error
[0.4858571 0.5141429]
Error
[0.47530988 0.5246901 ]
Error
[0.48939142 0.51060855]
Error
[0.48517585 0.51482415]
Error
[0.4849917  0.51500833]
Error
[0.50355095 0.49644908]
Error
[0.50461227 0.49538767]
Error
[0.4614916 0.5385084]
Error
[0.47994664 0.5200533 ]
Error
[0.47787607 0.52212393]
Error
[0.48861647 0.51138353]
Error
[0.4824958 0.5175042]
Error
[0.491163 0.508837]
Error
[0.5177966 0.4822034]
Error
[0.4775474  0.52245253]
Error
[0.5134581  0.48654196]
Error
[0.5267764 0.4732236]
Error
[0.46481377 0.5351863 ]
Error
[0.49690384 0.5030962 

In [7]:
import glob
from pathlib import Path
from typing import Dict


def get_true_labels(path: str) -> Dict[str, int]: 
    positive_label_files = [Path(file).stem for file in glob.glob(f"{path}/1/*")]
    negative_label_files = [Path(file).stem for file in glob.glob(f"{path}/0/*")]

    file_dicionary = {file : 1 for file in positive_label_files}
    file_dicionary.update({file : 0 for file in negative_label_files})

    return file_dicionary

def accuracy(model : CNNClassifier, path: str) -> float:
    true_labels = get_true_labels(path)
    predictions = model.predict(path)

    correct = 0
    for file in true_labels:
        prediction = predictions[file]
        if true_labels[file] == prediction:
            correct += 1
    return correct / len(true_labels)


def f1_score(model : CNNClassifier, path: str) -> float:
    true_labels = get_true_labels(path)
    predictions = model.predict(path)

    tp = 0
    fp = 0
    fn = 0
    for file in true_labels:
        prediction = predictions[file]
        if true_labels[file] == 1 and prediction == 1:
            tp += 1
        elif true_labels[file] == 1 and prediction == 0:
            fn += 1
        elif true_labels[file] == 0 and prediction == 1:
            fp += 1

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * (precision * recall) / (precision + recall)

In [8]:
accuracy(model, '../data/tsrc_sprectrograms/test')
f1_score(model, '../data/tsrc_sprectrograms/test')

0.6075949367088608